In [ ]:
import torch
import wandb
import os

from bayesian.models import HorseshoeSSVS
from bayesian.inference import inference
from ESN.utils import run_esn

import pyro

In [ ]:
config = {
            "dataset": "acea",
            "model_widths": [20, 10, 1],
            "activation": "tanh",
            "distributions": ["gauss", "unif", "gauss"],
            "parameters": [[0,1],[0,10]],
            "dim_reduction": False,
            "inference": "mcmc"
            }

os.environ["WANDB_MODE"]="offline"
wandb.init(project="bayes_rc", config=config)
config = wandb.config

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]='0'

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

Run ESN on dataset and produce embeddings with or without dimensionality reduction according to `dim_reduction`

In [ ]:
Ytr, train_embedding, val_embedding, Yte, test_embedding = run_esn(config.dataset, dim_reduction=config.dim_reduction)

# Stochastic Search Variable Selection

Perform SSVS on embeddings with no dimensionality reduction.

In [ ]:
# Define the horseshoe SSVS model
# type = 'm' for 'manual' linear layer, type = 't' for torch.nn.linear
ssvs_model = HorseshoeSSVS(train_embedding.shape(0), 1, type='m', device=device)

pyro.render_model(model_args=(train_embedding[0], Ytr[0]), render_distributions=True, filename=None)

In [ ]:
# Run Markov chains and collect diagnostics
predictive, diagnostics = inference(config, ssvs_model, None, X_train=train_embedding, Y_train=Ytr, X_test=test_embedding, Y_test=Yte)